In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import glob
import os

import datetime
import time
import ast

import itertools
import pickle

import requests
import re
import json
import bs4
import googlemaps

import cfscrape

# Column Alignment across different datasets
## Manual column renaming

In [14]:
country = 'Indonesia'

In [15]:
# define folder names
output_folder = 'outputs'
raw_outputs_folder = 'z raw'
cleaning_folder = "cleaning"
cleaned_outputs_folder = 'cleaned1'

In [16]:
# date today to mark date scraped
date_today = datetime.date.today().isoformat()

In [5]:
cou_file_list = glob.glob(os.path.join(output_folder, country,'*.csv'))

In [6]:
cou_file_list

['outputs\\Indonesia\\2018-08-31 - Corporate Accelerators (accelerators) - Indonesia.csv',
 'outputs\\Indonesia\\2018-08-31 - GALI (accelerators) - Indonesia.csv',
 'outputs\\Indonesia\\2018-08-31 - Google Places API, Nearby Places search (establishments) - Indonesia.csv',
 'outputs\\Indonesia\\2018-08-31 - Google Places API, Text search (accelerators, hubs, startups, companies) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - 1000 Startup Digital (mentors, entrepreneurs) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - Endeavor (companies, entrepreneurs) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - GBG Indonesia (companies) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - Indonesian ICT Society (MASTEL) (companies, associations) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - iPrice (companies) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-06 - [PARTIAL] Companiesss (companies) - Indonesia.csv',
 'outputs\\Indonesia\\2018-09-07 - Compass List Directory (foun

In [7]:
sample_data = {}
for file_name in cou_file_list:
    short_filename = file_name.split('\\')[-1].split('-')[-2].strip()
    sample_data[short_filename] = pd.read_csv(file_name, nrows = 1).T.fillna('NULL').to_dict()[0]

In [20]:
df_orig = pd.DataFrame(sample_data)

In [8]:
df_orig.to_csv(os.path.join(cleaning_folder,'%s - column alignment for %s.csv' % (date_today, country)))

## Merge in NULL columns and new datasets

In [25]:
glob.glob(os.path.join(cleaning_folder, '*.csv'))

['cleaning\\2018-09-12 - column alignment for Indonesia - MANUAL DRAFT.csv',
 'cleaning\\2018-09-12 - column alignment for Indonesia UPDATED.csv',
 'cleaning\\2018-09-12 - column alignment for Indonesia.csv']

In [13]:
df_updated = pd.read_csv('cleaning\\2018-09-12 - column alignment for Indonesia - MANUAL DRAFT.csv')

In [15]:
df_updated.columns = ['', 'Corporate Accelerators (accelerators)',
       'GALI (accelerators)',
       'Google Places API, Nearby Places search (establishments)',
       'Google Places API, Text search (accelerators, hubs, startups, companies)',
       '1000 Startup Digital (mentors, entrepreneurs)',
       'Endeavor (companies, entrepreneurs)', 'GBG Indonesia (companies)',
       'Indonesian ICT Society (MASTEL) (companies, associations)',
       'iPrice (companies)', '[PARTIAL] Companiesss (companies)',
       'Compass List Directory (founders)',
       'Compass List Directory (investors)',
       'Compass List profiles (startups)', 'Startup Ideas (founders)',
       'Startup Ideas (startups)', 'Private Banking',
       'Startup Ranking (founder)', 'Startup Ranking (investor)',
       'Startup Ranking (startup)']

In [18]:
df_updated = df_updated.set_index('')

In [22]:
df_updated.combine_first(df_orig).to_csv(os.path.join(cleaning_folder,'%s - column alignment for %s UPDATED.csv' % (date_today, country)))

## Renaming dataset columns

In [245]:
def extract_entity_type_private_banking(x):
    entity_type = 'financial institution'
    if 'banks' in x:
        entity_type = 'bank'
    elif 'venture capital' in x:
        entity_type = 'VC'
    elif 'angel investors' in x:
        entity_type = 'angel investor'
    elif 'associations' in x:
        entity_type = 'association'
    return(entity_type)

In [246]:
glob.glob(os.path.join(cleaning_folder, '*.csv'))

['cleaning\\2018-09-12 - column alignment for Indonesia - MANUAL DRAFT.csv',
 'cleaning\\2018-09-12 - column alignment for Indonesia FINAL Round 1.csv',
 'cleaning\\2018-09-12 - column alignment for Indonesia UPDATED.csv',
 'cleaning\\2018-09-12 - column alignment for Indonesia.csv',
 'cleaning\\2018-09-12 - entity_type assignment per data_source.csv']

In [254]:
# get list of files for cleaning and editing
cleaning_file_list = glob.glob(os.path.join(cleaning_folder, country, 'original','*.csv'))

In [255]:
# get cleaned, final columns per data source
df_cleaned_cols = pd.read_csv('cleaning\\2018-09-12 - column alignment for Indonesia FINAL Round 1.csv').set_index('Unnamed: 0')
cleaned_cols_dict = {}
data_source_list = df_cleaned_cols.columns
for data_source in data_source_list:
#     data_source_file = [file for file in cleaning_file_list if data_source in file][0]
    cleaned_cols_dict[data_source] = df_cleaned_cols[data_source].replace('DROP', np.nan).dropna().to_dict()

In [256]:
entity_type_dict = pd.read_csv('cleaning\\2018-09-12 - entity_type assignment per data_source.csv').set_index('filename').to_dict()['entity_type']

In [262]:
df_consolidated = pd.DataFrame()
for data_source, cleaned_cols in cleaned_cols_dict.items():
    # rename df temp columns
    file_name = [file for file in cleaning_file_list if data_source in file][0]
    if 'Endeavor' in data_source:
        df_temp = pd.read_csv(file_name)
    else:
        df_temp = pd.read_csv(file_name, usecols=list(cleaned_cols.keys()))
        df_temp = df_temp.rename(columns = cleaned_cols)
        
        # add temporary names for IDEA
        if data_source == 'IDEA (company)':
            df_temp['entity_name'] = df_temp['entity_url_website'].apply(lambda x: x.split('www.')[-1].split('//')[-1].split('.')[0])
    
    # add entity type if missing
    if 'entity_type' not in df_temp.columns:
        if data_source == 'Private Banking':
            df_temp['entity_type'] = df_temp['entity_type_list_page'].apply(lambda x: extract_entity_type_private_banking(ast.literal_eval(x)))
        else:
            entity_type = entity_type_dict[data_source]
            df_temp['entity_type'] = entity_type
    df_temp['entity_type'] = df_temp['entity_type'].str.lower()
    # add scraping metadata
    df_temp['entity_data_source'] = data_source 
    df_temp['entity_data_date_scraped_text'] = file_name.split("\\")[-1].split(' - ')[0]
    df_temp['entity_data_date_scraped_datetime'] = pd.to_datetime(file_name.split("\\")[-1].split(' - ')[0])

    # save cleaned files
    file_name_csv = file_name.split('\\')[-1]
    df_temp.to_csv(os.path.join(cleaning_folder, country, cleaned_outputs_folder,file_name_csv), index=False)
    
#     print(data_source, df_temp.shape)
    df_consolidated = df_consolidated.append(df_temp)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [263]:
df_consolidated.describe()

,entity_acceptance_rate,entity_address_colloquial_area,entity_address_district_postal_code,entity_admitted,entity_applicants,entity_contact_whatsapp,entity_desc_comment,entity_desc_cvc,entity_industry_naics_code,entity_industry_naics_sector,...,entity_web_moz_domain auth,entity_web_moz_page auth,entity_web_sr traffic - month_views/visit,entity_web_sr traffic - month_visits,entity_web_startupranking_country_rank,entity_web_startupranking_sr_score,entity_web_startupranking_sr_social,entity_web_startupranking_sr_web,entity_web_startupranking_world_rank,entity_year_of_establishment
count,0.0,2.000000,517.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1601.000000,1601.000000,1601.000000,1.601000e+03,1580.000000,1580.000000,1601.000000,1601.000000,1580.000000,311.000000
mean,NaN,0.500000,31289.085106,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,19.858214,22.253592,1.989694,2.772597e+05,790.500000,20184.987342,9932.695815,21436.586508,14180.546203,1978.864952
std,NaN,0.707107,33238.411788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.111322,11.956877,2.136139,3.366018e+06,456.251027,21348.136120,15197.728139,22785.163752,8142.999552,18.215938
min,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,2.000000,1.000000,0.900000,1.000000e+02,1.000000,153.000000,0.000000,170.000000,23.000000,1849.000000
25%,NaN,0.250000,12710.000000,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,9.000000,12.000000,1.000000,1.000000e+02,395.750000,2010.000000,265.000000,1870.000000,7337.500000,1964.000000
50%,NaN,0.500000,15820.000000,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,17.000000,21.000000,1.230000,4.240000e+02,790.500000,15942.500000,1857.000000,17181.000000,13689.500000,1982.000000
75%,NaN,0.750000,52417.000000,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,28.000000,31.000000,2.020000,1.675300e+04,1185.250000,32634.000000,14200.000000,35433.000000,21561.000000,1994.000000
max,NaN,1.000000,555281.000000,NaN,NaN,NaN,NaN,NaN,519130.0,51.0,...,91.000000,61.000000,30.090000,1.123918e+08,1580.000000,87381.000000,82645.000000,89063.000000,27455.000000,2009.000000


In [264]:
df_consolidated.to_csv(os.path.join(cleaning_folder, country, "%s - Consolidated Entity Data - %s.csv" % (date_today, country)), index=False)

In [265]:
df_consolidated['entity_type'].value_counts()

company                   27123
startup                    2315
founder                     581
university                  312
entrepreneur                278
mentor                      197
financial institution       186
investors                    63
bank                         54
association                  27
financial institutions       24
investor                     21
accelerator                  13
incubator                    13
association partner           9
vc                            4
business                      2
hub                           1
Name: entity_type, dtype: int64

# Creating relationship tables

In [266]:
glob.glob(os.path.join(cleaning_folder, country, cleaned_outputs_folder,'*.csv'))

['cleaning\\Indonesia\\cleaned1\\2018-08-31 - Corporate Accelerators (accelerators) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-08-31 - GALI (accelerators) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-08-31 - Google Places API, Nearby Places search (establishments) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-08-31 - Google Places API, Text search (accelerators, hubs, startups, companies) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - 1000 Startup Digital (mentors, entrepreneurs) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - Endeavor (company) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - Endeavor (founder) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - GBG Indonesia (companies) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - Indonesian ICT Society (MASTEL) (companies, associations) - Indonesia.csv',
 'cleaning\\Indonesia\\cleaned1\\2018-09-06 - iPrice (companies) - I

## compasslist

In [267]:
df_compasslist_startups = pd.read_csv('cleaning\\Indonesia\\cleaned1\\2018-09-07 - Compass List profiles (startups) - Indonesia.csv')

In [271]:
df_compasslist_startups = df_compasslist_startups[['entity_name', 'entity_investor_list']].set_index('entity_name')

In [278]:
dict_compasslist_startups = df_compasslist_startups['entity_investor_list'].apply(lambda x: ast.literal_eval(x)).to_dict()

In [280]:
list_compasslist_startups = []
for key, item in dict_compasslist_startups.items():
    for ite in item:
        list_compasslist_startups.append([key, ite])

In [293]:
df_compasslist_startups_rel = pd.DataFrame(list_compasslist_startups, columns=['destination_entity_name','source_entity_name'])
df_compasslist_startups_rel['source_entity_type'] = 'investor'
df_compasslist_startups_rel['relationship_type'] = 'is_investor_of'
df_compasslist_startups_rel['entity_data_source'] = 'Compass List profiles (investors)'

In [296]:
df_compasslist_startups_rel.to_csv(os.path.join(cleaning_folder, country, 'cleaned1rel','2018-09-07 - Compass List profiles (investors) - Indonesia.csv'), index=False)

## Inc42

In [3]:
df_inc42 = pd.read_csv('cleaning\\Indonesia\\cleaned1\\2018-09-12 - Inc42 (accelerator, incubator) - Indonesia.csv')

In [4]:
key_cols = ['entity_name','entity_type', 'entity_data_source', 'entity_funding_list']

In [5]:
df_inc42 = df_inc42[key_cols]
df_inc42 = df_inc42[df_inc42['entity_funding_list'].notnull()]

In [6]:
# remove and
df_inc42['entity_funding_list'] = df_inc42['entity_funding_list'].str.replace("and ","")

In [7]:
dict_inc42 = df_inc42.set_index('entity_name')['entity_funding_list'].apply(lambda x: x.split(', ')).to_dict()

In [8]:
list_inc42 = []
for key, item in dict_inc42.items():
    for ite in item:
        list_inc42.append([key, ite])

In [9]:
df_inc42_rel = pd.DataFrame(list_inc42, columns=['source_entity_name', 'destination_entity_name']).merge(df_inc42.drop(['entity_funding_list'], axis=1),
                                                                                         how='left',
                                                                                         left_on='source_entity_name',
                                                                                         right_on='entity_name').drop(['entity_name'],axis=1)

In [10]:
df_inc42_rel['relationship_type'] = 'is_investor_of'
df_inc42_rel['destination_entity_type'] = 'startup'

In [11]:
df_inc42_rel.columns = ['source_entity_name', 'destination_entity_name', 'source_entity_type',
       'entity_data_source', 'relationship_type', 'destination_entity_type']

In [12]:
df_inc42_rel = df_inc42_rel[['source_entity_name', 'source_entity_type','destination_entity_type',  'destination_entity_name', 'relationship_type','entity_data_source']]

In [17]:
df_inc42_rel.to_csv(os.path.join(cleaning_folder, country, 'cleaned1rel','2018-09-12 - Inc42 (accelerator, incubator) - Indonesia.csv'), index=False)

## Startup ranking

In [18]:
df_startupranking = pd.read_csv('cleaning\\Indonesia\\cleaned1\\2018-09-11 - Startup Ranking (startup) - Indonesia.csv')

In [19]:
df_startupranking = df_startupranking[['entity_name','entity_investor_list']].set_index('entity_name')

In [20]:
df_startupranking['entity_investor_list'] = df_startupranking['entity_investor_list'].apply(lambda x: ast.literal_eval(x))

In [21]:
df_startupranking['entity_investor_list_len'] = df_startupranking['entity_investor_list'].apply(lambda x: len(x))

In [22]:
df_startupranking = df_startupranking[df_startupranking['entity_investor_list_len'] > 0]

In [23]:
df_startupranking['entity_investors'] = df_startupranking['entity_investor_list'].apply(lambda x: ('\n'.join([val['Investors'] for val in x if val['Investors']!= ""])).split('\n'))
# .apply(lambda x: [val['Investor'] for val in x], axis=0)

In [24]:
df_startupranking['entity_investors_len'] = df_startupranking['entity_investors'].apply(lambda x: len(x) if x[0] != '' else 0)

In [25]:
df_startupranking = df_startupranking[df_startupranking['entity_investors_len'] > 0]

In [26]:
dict_sr = df_startupranking['entity_investors'].to_dict()

In [27]:
list_sr = []
for key, item in dict_sr.items():
    for ite in item:
        list_sr.append([key, ite])

In [28]:
df_sr_rel = pd.DataFrame(list_sr, columns = ['destination_entity_name', 'source_entity_name'])

In [29]:
df_sr_rel['source_entity_type'] = 'investor'
df_sr_rel['destination_entity_type'] = 'startup'
df_sr_rel['relationship_type'] = 'is_investor_of'
df_sr_rel['entity_data_source'] = 'Startup Ranking (startup)'

In [30]:
df_sr_rel = df_sr_rel[['source_entity_name', 'source_entity_type','destination_entity_type',  'destination_entity_name', 'relationship_type','entity_data_source']]

In [32]:
df_sr_rel.to_csv(os.path.join(cleaning_folder, country, 'cleaned1rel','2018-09-11 - Startup Ranking (startup) - Indonesia.csv'), index=False)

## combine all

In [33]:
cleaned_rel_list = glob.glob(os.path.join(cleaning_folder, country, 'cleaned1rel','*.csv'))

In [34]:
df_consolidated_rel = pd.DataFrame()
for file_name in  cleaned_rel_list:
    df_temp = pd.read_csv(file_name, encoding='latin-1')
    df_consolidated_rel = df_consolidated_rel.append(df_temp, ignore_index=True)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [35]:
df_consolidated_rel.to_csv(os.path.join(cleaning_folder, country, "%s - Consolidated Entity Relationship Data - %s.csv" % (date_today, country)), index=False)

In [38]:
df_consolidated_rel['relationship_type'].value_counts(dropna=False)

is_member_of        1414
is_founder_of       1056
is_investor_of       607
is_mentor_for        197
is_subsidiary_of       1
Name: relationship_type, dtype: int64